In [1]:
import pandas as pd
from datetime import datetime, timedelta
from iexfinance.stocks import get_historical_data
import iexfinance as iex
from iexfinance.refdata import get_symbols

In [2]:
# Use the `get_symbols` function from the iexfinance SDK to fetch available stock tickers
tickers=pd.DataFrame(get_symbols())
tickers.head()

,currency,date,exchange,iexId,isEnabled,name,region,symbol,type
0,USD,2019-09-18,NYS,IEX_46574843354B2D52,True,Agilent Technologies Inc.,US,A,cs
1,USD,2019-09-18,NYS,IEX_4238333734532D52,True,Alcoa Corp.,US,AA,cs
2,USD,2019-09-18,PSE,IEX_474B433136332D52,True,Perth Mint Physical Gold ETF,US,AAAU,et
3,USD,2019-09-18,NAS,IEX_4E5434354A302D52,True,Altaba Inc.,US,AABA,cef
4,USD,2019-09-18,NYS,IEX_4843364642592D52,True,AAC Holdings Inc.,US,AAC,cs


In [13]:
list_of_tickers = ['APHA', 'CRBP', 'CRON', 'GWPH', 'HEXO', 'TLRY', 'TRST', 'MJ']
# Set start and end datetimes of 1 year, between now and 365 days ago.
end_date = datetime.now()
start_date = end_date + timedelta(-365)

# Get 1 year's worth of historical data for "SPY" and "AGG"
df = get_historical_data(list_of_tickers, start_date, end_date,  output_format='pandas')
df.head()

APHA                                CRBP                    \
             open   high    low  close   volume  open  high   low close   
date                                                                      
2018-09-18  15.66  16.17  15.22  16.00  2750233  5.00  5.25  4.85  5.10   
2018-09-19  16.37  16.66  14.33  14.99  4916190  5.15  5.25  5.10  5.18   
2018-09-20  15.31  15.85  14.43  15.45  2594738  5.35  8.35  5.23  7.95   
2018-09-21  15.13  15.40  14.35  15.31  1646443  7.30  7.35  6.35  6.95   
2018-09-24  15.00  15.80  14.74  15.40  1255769  7.00  7.30  6.55  7.25   

                      ...  TRST                               MJ         \
              volume  ...  open  high   low close  volume   open   high   
date                  ...                                                 
2018-09-18   1105711  ...  8.80  8.80  8.55  8.55  316877  38.56  41.50   
2018-09-19    653329  ...  8.55  8.65  8.50  8.50  783641  43.65  45.40   
2018-09-20  18456419  ...  8.50  8.60  8.50  8.60  359617  42.55  43.69   
2018-09-21   9276121  ...  8.60  8.68  8.55  8.65  689264  42.70  42.86   
2018-09-24   2507502  ...  8.60  8.65  8.50  8.50  248537  38.32  39.86   

                                   
              low  close   volume  
date                               
2018-09-18  38.25  39.68  2468893  
2018-09-19  38.25  41.21  5218589  
2018-09-20  41.21  43.01  2471716  
2018-09-21  39.23  39.50  2412714  
2018-09-24  37.35  39.27  1311741  

[5 rows x 40 columns]

In [14]:
## Cleaning the Data
df.drop(columns=['open', 'high', 'low', 'volume'], level=1, inplace=True)
df.head()

,APHA,CRBP,CRON,GWPH,HEXO,TLRY,TRST,MJ
,close,close,close,close,close,close,close,close
date,,,,,,,,
2018-09-18,16.00,5.10,11.56,149.20,6.73,154.98,8.55,39.68
2018-09-19,14.99,5.18,12.69,153.64,6.34,214.06,8.50,41.21
2018-09-20,15.45,7.95,13.75,170.10,6.57,176.35,8.60,43.01
2018-09-21,15.31,6.95,12.51,164.98,6.52,123.00,8.65,39.50
2018-09-24,15.40,7.25,11.86,166.52,6.52,99.50,8.50,39.27


In [8]:
import os
os.getenv('IEX_TOKEN')

'pk_1e365e4ef8d942a5be9c5687d209a083'